In [1]:
import requests
import json
from datetime import datetime as dt

from storybot.models import Entry
from storybot.utils import upload_messages

## Step 1 - start the API server

The home directory, for the purposes of this tutorial, is the top level of this `lore-storybot` repo. To begin, 
`cd storybot` and run `uvicorn main:app --reload --port 8000`. This will start the primary API server. It utilizes some 
tools and models from Huggingface which may require a few minutes to download.  

## Step 2 - load the data 

Load the data. This application only used the `conversations` data set. 

In [2]:
file_dir = "./data"

# Load conversations
with open("/".join([file_dir, 'conversations.json']), 'r') as f:
    conversations = json.load(f)

## Step 3 - feature extraction and database upload

(See the README for detailed explanation.)

In [5]:
# Toggle verbosity to see features and DB upload responses
verbose = True
# verbose = False

feature_url = "http://127.0.0.1:8000/extract_sentiment"
entry_url = "http://127.0.0.1:8000/entry"

num_messages = 3
headers = {
    "Content-Type": "application/json"
}

# Mimic a streaming conversation
print("Extracting features and upload messages...")
for i, conv in enumerate(conversations[:num_messages]):
    print(f"\tmessage {i}")
    for message in conv['messages_list']:
        # Inefficient multiple lookup
        user_id = message['ref_user_id']
        conversation_id = message['ref_conversation_id']
        screen_name = message['screen_name']

        # skip the StoryBot responses
        if message['ref_user_id'] == 1:
            continue
        
        payload = {
            "message": message['message'],
            "metadata": {
                "user_id": user_id,
                "conversation_id": conversation_id,
                "screen_name": screen_name,
                "timestamp": message['transaction_datetime_utc']
            }
        }
        
        try:
            feature_response = requests.post(feature_url, headers=headers, json=payload)
        
            # Check if the request was successful (status code 2xx)
            if feature_response.status_code >= 200 and feature_response.status_code < 300:
                if verbose:
                    print("API Response:")
                    print(json.dumps(feature_response.json(), indent=2))
            else:
                print(f"Error: API request failed with status code {response.status_code}")
                print(f"Response content: {feature_response.text}")
                
        except requests.exceptions.RequestException as e:
            print(f"An error occurred during the request: {e}")
    
        try:
            item = Entry(**feature_response.json())
            db_response = requests.post(entry_url, headers=headers, data=item.model_dump_json())
          
            if db_response.status_code >= 200 and db_response.status_code < 300:
                if verbose:
                    print("DB API Response:")
                    print(json.dumps(db_response.json(), indent=2))
            else: 
                print(f"Error: DB API request failed with status code {db_response.status_code}")
                print(f"Response content: {db_response.text}")
                
        except requests.exceptions.RequestException as e:
            print(f"An error occurred during the request: {e}")

print("Finished!")

Extracting features and upload messages...
	message 0
API Response:
{
  "response": {
    "vader_scores": {
      "neg": 0.065,
      "neu": 0.819,
      "pos": 0.116,
      "compound": 0.296
    },
    "top_sentiment": {
      "label": "sadness",
      "score": 0.4427511990070343
    },
    "summary": {
      "summary_text": "Hello StoryBot, I\u2019m having a tough time with this app. My fingers aren\u2019t what they used to be. Can you help me?"
    },
    "entities": {
      "hands": "fingers"
    },
    "metadata": {
      "user_id": 782,
      "conversation_id": 98696,
      "screen_name": "ChattyPenguin",
      "timestamp": "2023-10-01T10:15:00Z"
    }
  }
}
DB API Response:
{
  "response": {
    "id": 104
  }
}
API Response:
{
  "response": {
    "vader_scores": {
      "neg": 0.184,
      "neu": 0.816,
      "pos": 0.0,
      "compound": -0.5255
    },
    "top_sentiment": {
      "label": "surprise",
      "score": 0.7161477208137512
    },
    "summary": {
      "summary_text

## Bonus! 

This is a parallel implemenation of the above feature extraction and upload steps. It's likely a little bit faster, but I wanted to leave the serial one intact since it's easier to discuss.

In [ ]:
# Uncomment to test this step
# upload_messages(conversations, num_conversations=10)

## Step 4 - feature analysis

Since end user -- different teams at Lore -- will want access to this data raw or processed data, 
the API also includes a feature analysis component. (See README for more details.)

In [18]:
user_id = 782

api_url = "http://127.0.0.1:8000/top-features"

headers = {
    "Content-Type": "application/json"
}

payload = {
    "user_id": user_id
}

resp = requests.post(api_url, headers=headers, json=payload)

In [23]:
# Top five unigrams or bigrams in the user's messages
resp.json()

{'response': {'able defend': 4.860729711040595,
  'able help': 4.860729711040595,
  'able home': 4.860729711040595,
  'able make': 4.860729711040595,
  'able offer': 4.860729711040595}}